엔터카는 중고차를 매매를 하는 회사이다. 엔터카 데이터 분석가는 중고 자동차에 대한 판매가격, 누적주행거리, 무게 등의 정보를 활용하여 자동차 사양에 따른 판매가격 예측모델을 만들고자 한다. 분석에 사용할 해당 데이터에 관한 정보는 아래와 같다.
1. enter_car_parti.txt : 자동차 사양 중 연속형 테이터 (1,004 Rows, 7 Columns) 
2. enter_car_part2.txt : 자동차 사양 중 범주형 데이터 (1,004 Rows, 4 Columns
각 컬럼의 정보는 다음과 같다. 

[enter_car_part1.txt]
- ID : 자동차ID, (Double)
- Price : 판매가격, (Double)
- Age : 차량 연식, (Double)
- KM : 누적 주행 거리, (Double)
- HP : 마력, (Double)
- CC : 배기량, (Double)
- Weight : 무게, (Double)

[enter_car_part2.txt]
- ID : 자동차ID (Double)
- FuelType : 사용 연료, (String)
- TransmissionType : 변속기 타입, (String)
- Doors : 문 개수, (Double)

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm

## Data read and preprocessing

In [8]:
data_continuous = pd.read_csv('enter_car_part1.txt')
data_discrete = pd.read_csv('enter_car_part2.txt')

In [9]:
print(type(data_continuous), data_continuous.shape)

<class 'pandas.core.frame.DataFrame'> (1004, 7)


In [11]:
print(type(data_discrete), data_discrete.shape)

<class 'pandas.core.frame.DataFrame'> (1004, 4)


#### 결측치 제거

In [14]:
np.sum(pd.isnull(data_continuous)), np.sum(pd.isnull(data_discrete))

(ID        0
 Price     0
 Age       0
 KM        0
 HP        0
 CC        0
 Weight    0
 dtype: int64,
 ID                  0
 FuelType            9
 TransmissionType    0
 Doors               0
 dtype: int64)

In [22]:
# 범주형 데이터에 결측치 제거
data_discrete_del_null = data_discrete.dropna(axis=0)

In [23]:
data_discrete_del_null

,ID,FuelType,TransmissionType,Doors
0,43.0,Petrol,Manual,2doors
1,44.0,Diesel,Manual,2doors
2,45.0,Diesel,Manual,5doors
3,46.0,Diesel,Manual,5doors
4,47.0,Diesel,Manual,5doors
...,...,...,...,...
999,974.0,Petrol,Manual,4doors
1000,984.0,Petrol,Manual,4doors
1001,986.0,Petrol,Automatic,4doors
1002,991.0,Petrol,Manual,4doors


Note: 분석에 앞서 다음 전처리를 수행하고 기본 데이터셋을 만드시오 
1. 데이터 A와 데이터 B를 자동차 ID를 기준으로 하나로 합친다 
2. 데이터에 결측이 존재하는 경우 삭제 
<주의 사항> 브라이틱스 이용자의 경우 결측임에도 공란 ()으로 인식하여 결측으로 여기지 않으므로 이를 발견하여 제거 필요

In [24]:
data = pd.merge(data_continuous, data_discrete_del_null, how='inner', on='ID')

In [25]:
data

,ID,Price,Age,KM,HP,CC,Weight,FuelType,TransmissionType,Doors
0,1,13500,23,46986,90,2000,1165,Diesel,Manual,4doors
1,2,13750,23,72937,90,2000,1165,Diesel,Manual,3doors
2,3,13950,24,41711,90,2000,1165,Diesel,Manual,4doors
3,4,14950,26,48000,90,2000,1165,Diesel,Manual,3doors
4,5,13750,30,38500,90,2000,1170,Diesel,Manual,4doors
...,...,...,...,...,...,...,...,...,...,...
990,1000,10950,57,40214,86,1300,1025,Petrol,Manual,4doors
991,1001,8950,60,40000,86,1300,1000,Petrol,Manual,4doors
992,1002,8950,66,39222,86,1300,1045,Petrol,Automatic,3doors
993,1003,8250,66,39144,110,1600,1050,Petrol,Manual,2doors


#### Doors 기준 평균 확인

In [30]:
grouped = data['Price'].groupby(data['Doors'])
grouped.mean()

Doors
2doors    12371.695431
3doors    11422.399061
4doors    11283.077193
5doors    12407.080000
Name: Price, dtype: float64

[문제 2-1]
변속기 타입(TransmissionType)이 판매가격에 영향을 주는지에 대한 통계적 검정을 진행하고자 한다. 적절한 검증 방법을 선택하여 유의수준 10%에서 검증하고 아래의 물음에 답하시오. 귀무가설 기각여부를 Y/N으로 기술, 분산 동질성 여부를 Y/N으로 기술하고 마지막으로 검정 통계량이 발생할 확률을 소수점 셋째 자리까지 작성하 시오(소수점 넷째 자리 미만은 버림)

In [33]:
data.TransmissionType, data.Price

(0         Manual
 1         Manual
 2         Manual
 3         Manual
 4         Manual
          ...    
 990       Manual
 991       Manual
 992    Automatic
 993       Manual
 994       Manual
 Name: TransmissionType, Length: 995, dtype: object,
 0      13500
 1      13750
 2      13950
 3      14950
 4      13750
        ...  
 990    10950
 991     8950
 992     8950
 993     8250
 994    10250
 Name: Price, Length: 995, dtype: int64)

In [35]:
type(data.TransmissionType[0])

str

In [32]:
t, p = stats.ttest_ind(data.TransmissionType, data.Price, equal_var=False)

TypeError: unsupported operand type(s) for /: 'str' and 'int'